<img src="./images/deepLearning1.png" align="left" width="1500"/>

<img src="./images/deepLearning2.png" align="left" width="1500"/>

<img src="./images/perceptron1.png" align="left" width="1500"/>

<img src="./images/perceptron2.png" align="left" width="1500"/>

<img src="./images/perceptron3.png" align="left" width="1500"/>

<img src="./images/perceptron4.png" align="left" width="1500"/>

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


퍼셉트론은 엄밀히 말하면 딥러닝은 아니지만 뉴런과 상당히 닮은 분류기이다.  
뉴런의 계산 과정을 이해하고 뉴런 하나만으로 AND, OR 연산과 같은 기초적인 연산을 할 수 있음을 알아본다.

In [2]:
# 퍼셉트론 코드의 이해도를 높이기 위해 1을 True로 0을 False로 그리고 편향(bias)은 1로 지정한다.
T = 1.0
F = 0.0
bias = 1.0

AND, OR, XOR 실습 데이터를 함수 형태로 정의해서 함수를 호출해서 데이터를 획득할 수 있게 한다.

In [3]:
def get_AND_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias]
    ] # 학습 데이터
    Y = [
        [F],
        [F],
        [F],
        [T]
    ] # 실제값, 레이블, 타겟, 클래스
    return X, Y

def get_OR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [T]]
    return X, Y

def get_XOR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [F]]
    return X, Y

In [4]:
# AND 연산에 필요한 데이터를 호출한다.
# X, Y = get_AND_data()
# OR 연산에 필요한 데이터를 호출한다.
# X, Y = get_OR_data()
# XOR 연산에 필요한 데이터를 호출한다.
X, Y = get_XOR_data()
X, Y

([[0.0, 0.0, 1.0], [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 1.0]],
 [[0.0], [1.0], [1.0], [0.0]])

텐서플로우를 사용해서 퍼셉트론을 구현한다.

In [5]:
# 논리 연산을 위한 입력값으로 X, Y, 편향값을 입력받아(총 3개의 입력) 1개의 값을 출력해야 하므로 가중치는 [3, 1]로 설정한다.
W = tf.Variable(tf.random_normal([3, 1])) # matmul() 메소드로 행렬의 곱셈을 실행하기 위해 3행 1열로 가중치를 만든다.
sess = tf.Session() # 세션을 만든다.
sess.run(tf.global_variables_initializer()) # 변수를 초기화시킨다.
print(sess.run(W))

[[0.25719664]
 [0.31543243]
 [1.9613006 ]]


퍼셉트론은 두 개의 입력이 있을 때 하나의 뉴런으로 두 개의 입력을 계산한 뒤 최종 결과값으로 0 또는 1을 출력한다.  
두 개의 입력은 가중치와 곱해져서 뉴런의 첫 번째 단계인 시그마로 들어가고 시그마 단계에서 모든 가중치 * 입력값과 편향값을 더하는 과정이 이루어진다.  
시그마 값은 뉴런의 두 번째 단계인 활성화 함수의 입력으로 들어가는데 퍼셉트론은 스텝 함수라는 활성화 함수를 사용한다. 스텝 함수는 시그마 값 >= 0 일 경우 1, 시그마 값이 < 0 일 경우 0을 출력한다.

In [6]:
# x가 0 이상이면 1, x가 0보다 작으면 0을 리턴하는 활성화 함수(스텝 함수)를 구현한다.
# tf.greater(a, b): a > b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.greater_equal(a, b): a >= b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.less(a, b): a < b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.less_equal(a, b): a <= b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.equal(a, b): a = b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.not_equal(a, b): a <> b일 경우 True, 그렇치 않으면 False를 리턴한다.
sess = tf.Session()
print(sess.run(tf.greater_equal(1, 0)))
f = tf.cast(tf.greater_equal(1, 0), dtype=tf.float32)
print(sess.run(f))

True
1.0


In [7]:
# 스텝 함수
def step(z):
    return tf.cast(tf.greater_equal(z, 0), dtype=tf.float32)

손실(오차, 비용) 함수로 평균 제곱 오차(MSE)를 사용한다.

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
z = tf.matmul(X, W) # 행렬 곱셈(내적) 메소드로 데이터와 가중치를 곱한다.
print(sess.run(z))
output = step(z)
print(sess.run(output))
# 레이블(실제값)과 스텝 함수의 실행 결과(예측값)의 편차를 계산한다.
error = tf.subtract(Y, output)
print(sess.run(error))
# 평균 제곱 오차를 계산한다.
mse = tf.reduce_mean(tf.square(error))
print(sess.run(mse))

[[-0.057927  ]
 [ 1.0057876 ]
 [-0.23297855]
 [ 0.83073604]]
[[0.]
 [1.]
 [0.]
 [1.]]
[[ 0.]
 [ 0.]
 [ 1.]
 [-1.]]
0.5


보통의 경우 딥 러닝 모델은 경사 하강법을 통해 매개 변수를 최적화하지만 미분값이 큰 의미가 없는 스텝 함수의 영향으로(입력값이 0일 때 미분 불가능, 입력값이 1일 때 미분값은 항상 0) 퍼셉트론은 아래와 같은 방법을 최적화를 진행한다.

결과값이 1이 나와야 하는데 스텝 함수의 출력이 0일 경우 => w_new = w_old + delta  
결과값이 0이 나와야 하는데 스텝 함수의 출력이 1일 경우 => w_new = w_old - delta  
위의 2가지 경우를 다음과 같이 사용할 수 있다.

In [9]:
import numpy as np

In [10]:
print(type(X))
print(type(np.array(X)))
print(np.array(X))
print(np.array(X).T) # 전치

<class 'list'>
<class 'numpy.ndarray'>
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
[[0. 0. 1. 1.]
 [0. 1. 0. 1.]
 [1. 1. 1. 1.]]


In [11]:
# 행렬의 transpose는 행과 열을 교환하여 얻는 전치행렬을 의미한다.
# matmul() 메소드의 transpose_a 옵션을 True로 지정하면 행렬의 내적을 계산하기 전에 1번째 행렬을 전치하고
# transpose_b 옵션을 True로 지정하면 행렬의 내적을 계산하기 전에 2번째 행렬을 전치한다.
delta = tf.matmul(X, error, transpose_a=True) # 행렬의 내적을 계산하기 전에 X를 전치한다.
print(sess.run(delta))
# assign() 메소드는 세션 안에서 텐서플로우 변수 값을 변경한다.
train = tf.assign(W, tf.add(W, delta)) # 역전파 => 가중치를 수정해서 다시 계산하기 위해서
print(sess.run(train))

[[ 0.]
 [-1.]
 [ 0.]]
[[-0.17505156]
 [ 0.06371462]
 [-0.057927  ]]


학습 후 테스트

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

err, epoch, max_epochs = 1.0, 0, 20 # 오차, 현재 학습 횟수, 최대 학습 횟수
# 오차가 발생하고 학습 횟수가 최대 학습 횟수를 넘어가지 않는 동안 반복한다.
while err > 0.0 and epoch < max_epochs:
    epoch += 1 # 학습 횟수를 증가시킨다.
    err = sess.run(mse) # 평균 제곱 오차를 계산한다.
    sess.run(train) # 학습시킨다.
    print('epoch: {:2d}, mse: {}'.format(epoch, err))
# ===== while

print('테스트 결과\n', sess.run(output), sep='')
sess.close()

epoch:  1, mse: 0.5
epoch:  2, mse: 0.5
epoch:  3, mse: 0.5
epoch:  4, mse: 0.5
epoch:  5, mse: 0.5
epoch:  6, mse: 0.5
epoch:  7, mse: 0.5
epoch:  8, mse: 0.5
epoch:  9, mse: 0.5
epoch: 10, mse: 0.5
epoch: 11, mse: 0.5
epoch: 12, mse: 0.5
epoch: 13, mse: 0.5
epoch: 14, mse: 0.5
epoch: 15, mse: 0.5
epoch: 16, mse: 0.5
epoch: 17, mse: 0.5
epoch: 18, mse: 0.5
epoch: 19, mse: 0.5
epoch: 20, mse: 0.5
테스트 결과
[[0.]
 [0.]
 [0.]
 [0.]]


In [14]:
# Session을 with 구문을 사용해서 만들면 with 블록이 종료되는 순간 자동으로 close 된다.
with tf.Session() as sess: # 세션을 만든다.
    sess.run(tf.global_variables_initializer())
    err, epoch, max_epochs = 1.0, 0, 20 # 오차, 현재 학습 횟수, 최대 학습 횟수
    # 오차가 발생하고 학습 횟수가 최대 학습 횟수를 넘어가지 않는 동안 반복한다.
    while err > 0.0 and epoch < max_epochs:
        epoch += 1 # 학습 횟수를 증가시킨다.
        err = sess.run(mse) # 평균 제곱 오차를 계산한다.
        sess.run(train) # 학습시킨다.
        print('epoch: {:2d}, mse: {}'.format(epoch, err))
    # ===== while
    print('테스트 결과\n', sess.run(output), sep='')
# ===== with

epoch:  1, mse: 0.5
epoch:  2, mse: 0.5
epoch:  3, mse: 0.5
epoch:  4, mse: 0.5
epoch:  5, mse: 0.5
epoch:  6, mse: 0.5
epoch:  7, mse: 0.5
epoch:  8, mse: 0.5
epoch:  9, mse: 0.5
epoch: 10, mse: 0.5
epoch: 11, mse: 0.5
epoch: 12, mse: 0.5
epoch: 13, mse: 0.5
epoch: 14, mse: 0.5
epoch: 15, mse: 0.5
epoch: 16, mse: 0.5
epoch: 17, mse: 0.5
epoch: 18, mse: 0.5
epoch: 19, mse: 0.5
epoch: 20, mse: 0.5
테스트 결과
[[0.]
 [0.]
 [0.]
 [0.]]
